<a href="https://colab.research.google.com/github/matlafu/Reading_Time_Prediction/blob/main/Bert_base_german_cased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 13.3 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

# Load the German BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-german-cased')
model = AutoModelForMaskedLM.from_pretrained('bert-base-german-cased')

# Define the input sentence and the token you want to get the surprisal for
sentence = "Hans isst einen Baum."
token = "isst"

# Tokenize the input sentence
tokenized_sentence = tokenizer.tokenize(sentence)
print(f"Tokenized sentence:{tokenized_sentence}" )

# Find the index of the token in the tokenized sentence
token_index = tokenized_sentence.index(token)
print(f"Token Index: {token_index}")

# Mask the token in the tokenized sentence
masked_sentence = tokenized_sentence.copy()
masked_sentence[token_index] = tokenizer.mask_token
print(f"Masked Sentence: {masked_sentence}")

# Convert the tokenized sentence to input IDs
input_ids = tokenizer.convert_tokens_to_ids(masked_sentence)
print(f"Input ids: {input_ids}")

# Add a batch dimension to the input IDs tensor
input_ids = torch.tensor([input_ids])

# Get the model's log-softmax output for the input IDs
outputs = model(input_ids)
print(f"Outputs: {outputs}")

# Get the log-softmax output for the masked token
logits = outputs.logits[0, token_index]
print(f"Logits: {logits}")

log_prob = logits.softmax(dim=-1)[tokenizer.convert_tokens_to_ids(token)]
print(f"log_prob: {log_prob}")

# Calculate the surprisal as the negative log probability
surprisal = -log_prob.log()
print(f"Surprisal: {surprisal}")

print(f"Surprisal of '{token}' in '{sentence}': {surprisal:.2f}")



Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Tokenized sentence:['Hans', 'is', '##st', 'einen', 'Baum', '.']


ValueError: ignored

In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

# Load the German BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-german-cased')
model = AutoModelForMaskedLM.from_pretrained('bert-base-german-cased')

# Define the input sentence and the token you want to get the surprisal for
sentence = "Hans isst einen Apfel"
token = "isst"

# Tokenize the input sentence with the German BERT tokenizer
tokenized_sentence = tokenizer.tokenize(sentence)
print(f"Tokenized sentence:{tokenized_sentence}" )

# Find the indices of the subword tokens corresponding to the token
token_subwords = tokenizer.tokenize(token)
token_indices = [i for i, token in enumerate(tokenized_sentence) if token in token_subwords]
token_index_range = range(len(token_indices))
print(f"Token index range: {token_index_range}")
print(f"Token Index: {token_indices}")

# Convert the tokenized sentence to input IDs
input_ids = torch.tensor(tokenizer.convert_tokens_to_ids(tokenized_sentence))
print(f"Input ids: {input_ids}")

# Calculate the mask for the token subwords
mask = torch.zeros_like(input_ids)
for index in token_indices:
    mask[index] = 1
print(f"masked: {mask}")

# Get the model's log-softmax output for the masked input IDs
outputs = model(input_ids.unsqueeze(0), attention_mask=mask.unsqueeze(0))
print(f"Outputs: {outputs}")

# Take the ids for the tokens
token_ids = [input_ids[i] for i in token_indices]
print(f"token_ids: {token_ids}")

# Calculate the log-softmax probabilities for the token subwords
logits = outputs.logits[0, token_indices]
print(f"logits: {logits}")
log_probs = logits.softmax(dim=-1)
print(f"logprobs: {log_probs}")
# Calculate the surprisal as the negative log probability
surprisal = -log_probs.sum()

# dis one my take
#logits2 = [logits[0][i] for i in token_ids] # hier sollte ich auch nen for loop fuer token index machen cuz we got more than 1 tensorrrrrrr
zipped = zip(token_index_range,token_ids)
logits2 = [logits[x][d] for x,d in zipped]
print(f"logits2: {logits2}")
log_probs2 = logits2[0].softmax(dim=-1)
print(f"logprobs2: {log_probs2}")
surprisal2 = -log_probs2
print(f"Surprisal2: {surprisal2}")


print(f"Surprisal of '{token}' in '{sentence}': {surprisal:.2f}")


Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Tokenized sentence:['Hans', 'is', '##st', 'einen', 'Apfel']
Token index range: range(0, 2)
Token Index: [1, 2]
Input ids: tensor([ 2944,  4299,    13,   303, 23699])
masked: tensor([0, 1, 1, 0, 0])
Outputs: MaskedLMOutput(loss=None, logits=tensor([[[-6.9248, -4.3191,  8.3188,  ..., -4.4270, -4.1049, -4.9304],
         [-6.6925, -3.8285,  6.5143,  ..., -4.0112, -3.1632, -3.8015],
         [-7.2668, -4.8614,  8.0459,  ..., -4.7671, -5.0126, -4.9704],
         [-6.5043, -3.7077,  7.3376,  ..., -4.0655, -3.1707, -4.5095],
         [-6.9266, -4.3387,  8.7308,  ..., -4.4971, -4.2754, -4.9071]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)
token_ids: [tensor(4299), tensor(13)]
logits: tensor([[-6.6925, -3.8285,  6.5143,  ..., -4.0112, -3.1632, -3.8015],
        [-7.2668, -4.8614,  8.0459,  ..., -4.7671, -5.0126, -4.9704]],
       grad_fn=<IndexBackward0>)
logprobs: tensor([[4.6357e-11, 8.1270e-10, 2.5222e-05,  ..., 6.7704e-10, 1.5808e-09,
         8.3495e-10],
      